In [3]:
# Scrapy 시작할 폴더
%cd /Users/seokholee/lsh/Project/sol4/dc_crawl

/Users/seokholee/lsh/Project/sol4/dc_crawl


In [ ]:
# Scrapy Project Create
!scrapy startproject dc

In [ ]:
# Install fake-useragent
!pip install scrapy-fake-useragent

### 1. items.py 만들기

In [ ]:
%%writefile dc/dc/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class DcItem(scrapy.Item):
    title = scrapy.Field()
    content = scrapy.Field()
    date = scrapy.Field()
    views = scrapy.Field()
    recommend = scrapy.Field()
    link = scrapy.Field()
    gall_code = scrapy.Field()



### 2. spider 만들기
- time:sleep(1) 로 크롤링 차단 방지

In [ ]:
%%writefile dc/dc/spiders/spider.py

import scrapy
import time
from dc.items import DcItem
from dc.custom_fun import *

class DcSpider(scrapy.Spider):
    name = "dc"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES' : {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    def __init__(self, **kwargs):
        with open('/Users/seokholee/lsh/Project/sol4/dc_crawl/dc/dc/spiders/target_gall.txt', 'r') as f:
            gall_ids = f.readlines()
        
        self.url_lst = []
        for gall_id in gall_ids:
            gall_id = re.sub('\n', '', gall_id)
            self.url_lst.append(make_galleryURL(gall_id))
        
        super().__init__(**kwargs)
        
    def start_requests(self):

        # 10페이지 클롤링 / 페이지당 게시글 100개 총 1000개의 게시글 크롤링
        for url in self.url_lst:
            print('START CRAWL : ' + url)
            for page in range(1,2):
                gall_url = url + "&page=" + str(page) + "&list_num=100"
                yield scrapy.Request(gall_url, callback=self.parse, meta = { 'url' : url })
        
    def parse(self, response):
        links = response.xpath('//*[@class="ub-content us-post"]/td/a/@href').extract()
        links = list(map(response.urljoin, links))
        print('RESPONSE URLJOIN : ', links)
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse, meta={**response.meta})
    
    def page_parse(self, response):
        item = DcItem()
        item["title"] = response.xpath('//*[@class="gallview_head clear ub-content"]/h3/span[2]/text()').extract_first()
        
        contents = response.xpath('//*[@class="write_div"]/p/text()').extract()
        post = ''
        for content in contents:
            content = re.sub('[\n\t]', '', content)
            content += '\n'
            post += content
        item['content'] = post[:-2]

        item["date"] = response.xpath('//*[@class="gall_date"]/text()').extract_first()
        item["views"] = response.xpath('//*[@class="fr"]/span[1]/text()').extract_first()[3:]
        item["recommend"] = response.xpath('//*[@class="gall_reply_num"]/text()').extract_first()[3:]
        item["link"] = response.url
        
        url = response.meta.pop('url')
        item['gall_code'] = re.search('(?<=id=).+', url).group(0)
        
        yield item


### 3. setting.py 변경
- ROBOTSTXT_OBEY = False
- ITEM_PIPELINES = 300

In [ ]:
%%writefile dc/dc/settings.py


# Scrapy settings for comics project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'dc'

SPIDER_MODULES = ['dc.spiders']
NEWSPIDER_MODULE = 'dc.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'comics (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# 다운로드 딜레이
DOWNLOAD_DELAY = 0.5


### 4. 갤러리 입력

- dc인사이드 접속 -> 원하는 개럴리 접속
- 주소 중 ?id = " " <- id= 뒤에있는 코드 입력 
- 한줄에 갤러리 코드 하나씩 입력

In [2]:
football_new8
dnfqq
comic_new3


Overwriting dc/dc/spiders/target_gall.txt


### 5. 종료시간 입력
- 어느 날짜까지 크롤링할지 입력
- YYYY-mm-dd HH:MM:SS 형식 유지 (DC인사이드 시간 정보 포맷)

In [ ]:
%%writefile dc/dc/spiders/target_gall.txt
2023-08-08 15:09:33

In [4]:
# 이 명령어를 실행하면 위 코드 블록은 경로가 달라져서 실행이 안됨.
# Scrapy 실행을 위해 클롤러 위치로 이동
%cd /Users/seokholee/lsh/Project/sol4/dc_crawl/dc/dc/spiders

/Users/seokholee/lsh/Project/sol4/dc_crawl/dc/dc/spiders


In [ ]:
# 크롤러 실행 -o 저장할 파일이름.csv
!scrapy crawl dc -o dc_crawl.csv